In [46]:
import finetune_resnet
import h5py
from keras.metrics import sparse_categorical_accuracy
import numpy as np
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [34]:
dataset = h5py.File("../Market-1501/market.h5")
x = dataset["train_images"]
attributes_target = dataset["train_labels"]
file_names = dataset["train_images_files"]
ids_encoder = dict()
train_ids = np.zeros(len(file_names), dtype=int)
i = 0
for j, file in enumerate(file_names):
    if file[:4] not in ids_encoder:
        ids_encoder[file[:4]] = i
        i += 1
    train_ids[j] = ids_encoder[file[:4]]
n_id = len(ids_encoder)

In [40]:
train_ids.shape, n_id

((12567,), 751)

In [9]:
_, training_model, inference_model = finetune_resnet.get_models(n_person=n_id)














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
training_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 64, 3)   0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 2048)         23587712    input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 27)           55323       resnet50[1][0]                   
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 512)          1049088     resnet50[1][0]                   
__________________________________________________________________________________________________
dense_2 (D

In [17]:
# sparse_top_k_categorical_accuracy
# TODO: rename layers
training_model.compile("adam", loss=["binary_crossentropy", "sparse_categorical_crossentropy"], metrics={"attributes":[finetune_resnet.market_attribute_accuracy], "ids":[sparse_categorical_accuracy]})

In [20]:
attributes, categorical_id = training_model.predict(finetune_resnet.preprocess_images(x[:1]))
attributes.shape, categorical_id.shape

((1, 27), (1, 751))

In [43]:
id_embedding = inference_model.predict(finetune_resnet.preprocess_images(x[:1]))
id_embedding.shape

(1, 539)

In [49]:
cb = [
    ModelCheckpoint("../models/full-model-test-0.h5")
]

n_samples = 400
epochs = 5
batch_size = 32

history = training_model.fit(
    x=finetune_resnet.preprocess_images(np.array(x[:n_samples])),
    y=[finetune_resnet.preprocess_labels(np.array(attributes_target[:n_samples])), train_ids[:n_samples]],
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=cb,
)



Train on 320 samples, validate on 80 samples
Epoch 1/5
320/320 [==============================] - 160s 501ms/step - loss: 5.1758 - dense_1_loss: 0.4111 - dense_4_loss: 4.7647 - dense_4_sparse_categorical_accuracy: 0.3219 - val_loss: 8.7379 - val_dense_1_loss: 0.8512 - val_dense_4_loss: 7.8867 - val_dense_4_sparse_categorical_accuracy: 0.0125
Epoch 2/5
320/320 [==============================] - 87s 272ms/step - loss: 1.2620 - dense_1_loss: 0.1814 - dense_4_loss: 1.0806 - dense_4_sparse_categorical_accuracy: 0.8375 - val_loss: 9.2352 - val_dense_1_loss: 0.8606 - val_dense_4_loss: 8.3746 - val_dense_4_sparse_categorical_accuracy: 0.0125
Epoch 3/5
320/320 [==============================] - 64s 200ms/step - loss: 0.3756 - dense_1_loss: 0.0922 - dense_4_loss: 0.2834 - dense_4_sparse_categorical_accuracy: 0.9437 - val_loss: 9.2186 - val_dense_1_loss: 0.7889 - val_dense_4_loss: 8.4297 - val_dense_4_sparse_categorical_accuracy: 0.0125
Epoch 4/5
320/320 [==============================] - 59s 1

In [50]:
history.history

{'val_loss': [8.737919425964355,
  9.235214328765869,
  9.218600177764893,
  8.635454177856445,
  8.813895797729492],
 'val_dense_1_loss': [0.8512229681015014,
  0.8605713605880737,
  0.7888700842857361,
  0.5309891402721405,
  0.46463860273361207],
 'val_dense_4_loss': [7.886696434020996,
  8.374642944335937,
  8.429729843139649,
  8.104464721679687,
  8.349256896972657],
 'val_dense_4_sparse_categorical_accuracy': [0.0125,
  0.0125,
  0.0125,
  0.0,
  0.0125],
 'loss': [5.175798225402832,
  1.2619544088840484,
  0.37555939257144927,
  0.17835927978157998,
  0.10562902688980103],
 'dense_1_loss': [0.41110323965549467,
  0.18139218389987946,
  0.09216878935694695,
  0.04728953205049038,
  0.025378935039043427],
 'dense_4_loss': [4.7646949052810665,
  1.080562210083008,
  0.28339060246944425,
  0.13106974894180895,
  0.08025009389966727],
 'dense_4_sparse_categorical_accuracy': [0.321875,
  0.8375,
  0.94375,
  0.96875,
  0.975]}

In [60]:
test_data = np.array([
    x[0], x[1], x[2], x[100], x[200]
])
res = inference_model.predict(finetune_resnet.preprocess_images(test_data))

In [61]:
# 0 1 2 are the same person, 3, 4 are different persons
dist = [
    np.linalg.norm(res[0] - res[1]),
    np.linalg.norm(res[0] - res[2]),
    np.linalg.norm(res[0] - res[3]),
    np.linalg.norm(res[0] - res[4]),
]
dist

[8.696653, 7.113424, 33.922447, 44.89048]